In [5]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray
import seaborn as sns
from tqdm import tqdm
from joblib_progress import joblib_progress
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

In [6]:
# paths
high_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code15_n5.gpkg'
high_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code12_n5.gpkg'
un_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code3_n5.gpkg'
un_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code0_n5.gpkg'
poly_paths = [high_high_path, high_un_path, un_high_path, un_un_path]

helena_path = Path.cwd().parent / 'data' / 'helena'
crown_path = helena_path / 'crowns'
crown_path_list = [
    c for c
    in crown_path.iterdir()
    if c.suffix == '.gpkg'
    ]

# open treatment polygons
df = pd.concat([gpd.read_file(p) for p in poly_paths])
df = df.drop('area_', axis=1)

We will first check each crown to see if it falls completely within one of the treatment class areas.  If so it will be appended to a datframe of crowns.


In [7]:
# jobs to run in ||
n_jobs = 10

def label_treatment(f):
    crown_df = gpd.read_file(f)
    crown_df['treatment'] = -99

    for i, row in df.iterrows():
        crown_df.loc[
            crown_df.geometry.within(row.geometry),
            'treatment'] = row.attribute
        
    return crown_df[crown_df.treatment >= 0 ]

with joblib_progress('Labeling crowns by treatment...', total=len(crown_path_list)):
    results = Parallel(n_jobs=n_jobs)(
        delayed(label_treatment)(f)
        for f
        in crown_path_list
        )
    
crown_df = pd.concat(results)
            

Output()